# alternating blocked descent

- using SGD on $C$
- after each pass over the observed parts of the covariance matrix, use analyic solution for $A$, $\Pi$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import glob, os

os.chdir('../core')
import SSID_Hankel_loss 
from utility import get_subpop_stats, draw_sys, gen_data
from SSID_Hankel_loss import run_bad, plot_outputs_l2_gradient_test, l2_bad_sis_setup
os.chdir('../dev')

import psutil
import time

# load ancient code for drawing from LDS ...
os.chdir('../../../../pyRRHDLDS/core')
import ssm_scripts
import ssm_fit
os.chdir('../../code_le_stitch/iterSSID/python/dev')

#np.random.seed(0)

p,n,nr = 10000, 100, 50
k,l = 5,5

T = 5000
mmap = True
data_path = '../fits/'
chunksize = np.min((p,2000))
max_i = p//chunksize

eig_m_r, eig_M_r, eig_m_c, eig_M_c = 0.8, 0.99, 0.8, 0.99

verbose=True

# create subpopulations
sub_pops = (np.arange(0,p), np.arange(0,p))
#sub_pops = (np.arange(p//2), np.arange(p//2,p))

"""
len_sub_pops = 9                                            #
pop_size = p//5                                             # stochastic turn-over ('funky' observation scheme)
sub_pops = [np.arange(pop_size)]                            # 
for i in range(1,len_sub_pops):
    old = np.atleast_1d(np.random.choice(sub_pops[i-1],pop_size//2,replace=False))
    assert old.size == pop_size//2
    assert np.all([old[j] in sub_pops[i-1] for j in range(old.size)])
    new = np.arange((1/2+i/2)*pop_size,((i+1)/2+1/2)*pop_size,dtype=np.int)
    sub_pops.append(np.hstack((old,new)))
    assert len(sub_pops[-1]) == pop_size
"""
#sub_pops = (np.arange(0,200), np.arange(100,300), np.arange(200,400), np.arange(300,p))
#sub_pops = (np.arange(0,2*p//10), np.arange(p//10,3*p//10), np.arange(2*p//10,4*p//10), np.arange(3*p//10,5*p//10),
#            np.arange(4*p//10,6*p//10), np.arange(5*p//10,7*p//10), np.arange(6*p//10,8*p//10), 
#            np.arange(7*p//10,9*p//10), np.arange(8*p//10,p))
#sub_pops = (np.arange(0,p//2+1), np.arange(p//2-1,p))
#sub_pops = (np.arange(10,p), np.arange(0,p-10))

obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

# draw system matrices    
ev_r = np.linspace(eig_m_r, eig_M_r, nr)
ev_c = np.exp(2 * 1j * np.pi * np.random.uniform(size= (n - nr)//2))
ev_c = np.linspace(eig_m_c, eig_M_c, (n - nr)//2) * ev_c

calc_stats = True if T == np.inf else False
pars_true, Qs, Qs_full = draw_sys(p=p,n=n,k=k,l=l, nr=nr, ev_r=ev_r,ev_c=ev_c,calc_stats=calc_stats,
                                 return_masked=False, mmap=mmap, chunksize=chunksize, data_path=data_path)
pars_true['d'], pars_true['mu0'], pars_true['V0'] = np.zeros(p), np.zeros(n), pars_true['Pi'].copy()

#pars_true['R'] = 10e-5 * np.ones(p)
#pars_true['Q'] = 10e-10 * np.eye(n)

if calc_stats:
    x,y = np.zeros((n,0)), np.zeros((p,0))
else:
    print('computing empirical covariances')
    x,y = gen_data(pars = pars_true, T = T, mmap = mmap, chunksize = chunksize, data_path = data_path ) 
    for m in range(k+l):
        print('computing time-lagged covariance for lag ', str(m))
        if mmap:
            Q = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, mode='w+', shape=(p,p))
        else:
            Q = np.empty((p,p))
        for i in range(max_i):
            idx_i  = range(i*chunksize, (i+1)*chunksize)
            for j in range(max_i):
                idx_j = range(j*chunksize, (j+1)*chunksize)
                Q[np.ix_(idx_i,idx_j)] = np.cov(y[m:m-(k+l),idx_i].T, y[:-(k+l),idx_j].T)[:chunksize,chunksize:]     
        if mmap:
            del Q
            Qs_full[m] = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, mode='r', shape=(p,p))
        else:
            Qs_full[m] = Q
        Qs[m] = Qs_full[m]
        
os.chdir('../fits/')

save_file = 'test_time_Tinf_p10000n100r1'


np.savez(save_file, 
         pars_init=None,
         pars_true=pars_true, 
         pars_est=None,
         sub_pops=sub_pops,
         p=p,n=n,T=T,k=k,l=l)          
        
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())

#Qs[0] = None     
#Qs_full[0] = None



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.optimize import fmin_bfgs, check_grad
import glob, os

os.chdir('../core')
import SSID_Hankel_loss 
from utility import get_subpop_stats, draw_sys, gen_data
from SSID_Hankel_loss import run_bad, plot_outputs_l2_gradient_test, l2_bad_sis_setup
os.chdir('../dev')

import psutil
import time

# load ancient code for drawing from LDS ...
os.chdir('../../../../pyRRHDLDS/core')
import ssm_scripts
import ssm_fit
os.chdir('../../code_le_stitch/iterSSID/python/dev')


from scipy.io import savemat # store results for comparison with Matlab code   

os.chdir('../fits/')

save_file = np.load('test_time_Tinf_p10000n100r1.npz')
p,n,T,k,l = save_file['p'], save_file['n'], save_file['T'], save_file['k'], save_file['l']
pars_true = save_file['pars_true']
#pars_est, pars_init = save_file['pars_est'], save_file['pars_init']

Qs_full, Qs = [], []
for m in range(k+l):
    print('loading time-lagged covariance for lag ', str(m))
    Qs_full.append(np.memmap('Qs_'+str(m), dtype=np.float, mode='r', shape=(p,p)))
    Qs.append(Qs_full[m])

y = np.memmap('y'+str(m), dtype=np.float, mode='r', shape=(T,p))
sub_pops = (np.arange(p), np.arange(p))
obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

chunksize = 5000
max_i = p//chunksize
max_zip_size = 5000

verbose=True
            
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())


In [ ]:
batch_size = 1 # batch_size = 1 (size-1 mini-batches), p (column mini-batches), None (full gradients)
max_zip_size = p//100
max_iter_nl  = 10
max_iter_lin = 0

a, b1, b2, e = 0.1, 0.9, 0.99, 1e-8
lag_range  = 1



t = time.time()

linearity = 'False'
stable = False
sym_psd = False
pars_init, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,Qs_full=None,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      linearity=linearity,stable=stable,init='default',
                                      alpha=a,b1=b1,b2=b2,e=e,max_iter=max_iter_nl,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, lag_range = lag_range, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t)

print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())

pars = pars_est
idx1 = np.random.choice(p, np.min((p,1000)), replace=False)
idx2 = np.random.choice(p, np.min((p,1000)), replace=False)
plt.figure(figsize=(20,50))
for m in range(0,k+l): 
    Qrec = pars['C'][idx1,:].dot(pars['X'][m*n:(m+1)*n, :]).dot(pars['C'][idx2,:].T) 
    Qrec = Qrec + np.diag(pars['R'])[np.ix_(idx1,idx2)] if m==0 else Qrec
    
    plt.subplot(np.ceil( (k+l)/2 ), 2, m+1, adjustable='box-forced')

    if mmap:
        Q = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, mode='r', shape=(p,p))
    else:
        Q = Qs[m]
        
    plt.plot(Q[np.ix_(idx1,idx2)].reshape(-1), Qrec.reshape(-1), '.')
    if m == 0:
        plt.hold(True)
        idx12 = np.intersect1d(idx1,idx2)
        plt.plot(Q[idx12,idx12], np.diag(pars['C'][idx12,:].dot(pars['X'][m*n:(m+1)*n, :]).dot(pars['C'][idx12,:].T)), 'r.')
    plt.title( ('m = ' + str(m) + ', corr = ' + 
    str(np.corrcoef( Qrec.reshape(-1), (Qs[m][np.ix_(idx1,idx2)]).reshape(-1) )[0,1])) )
    
    if mmap:
        del Q
        
    plt.xlabel('true covs')
    plt.ylabel('est. covs')
plt.show()
    
plt.figure(figsize=(20,10))
plt.plot(traces[0])
plt.xlabel('iteration count')
plt.ylabel('target loss')
plt.title('loss function vs. iterations')
plt.show()

if p <= 1000:
    plot_outputs_l2_gradient_test(pars_true=pars_true, pars_init=pars_init, 
                                  pars_est=pars_est, k=k, l=l, Qs=Qs, 
                                       Qs_full=Qs, Om=Om, traces = traces,
                                       linearity=linearity, idx_grp = idx_grp, co_obs = co_obs, 
                                       if_flip = True, m = 0)

In [ ]:
batch_size = 100 # batch_size = 1 (size-1 mini-batches), p (column mini-batches), None (full gradients)
max_iter_nl  = 3
max_iter_lin = 0

t = time.time()

linearity = 'False'
stable = False
sym_psd = False
_, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,Qs_full=None,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      linearity=linearity,stable=stable,init=pars_est,
                                      alpha=a,b1=b1,b2=b2,e=e,max_iter=max_iter_nl,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, lag_range = lag_range, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t)

print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())

pars = pars_est
idx1 = np.random.choice(p, np.min((p,1000)), replace=False)
idx2 = np.random.choice(p, np.min((p,1000)), replace=False)
plt.figure(figsize=(20,50))
for m in range(0,k+l): 
    Qrec = pars['C'][idx1,:].dot(pars['X'][m*n:(m+1)*n, :]).dot(pars['C'][idx2,:].T) 
    Qrec = Qrec + np.diag(pars['R'])[np.ix_(idx1,idx2)] if m==0 else Qrec
    
    plt.subplot(np.ceil( (k+l)/2 ), 2, m+1, aspect='equal')

    if mmap:
        Q = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, mode='r', shape=(p,p))
    else:
        Q = Qs[m]
        
    plt.plot(Q[np.ix_(idx1,idx2)].reshape(-1), Qrec.reshape(-1), '.')
    plt.title( ('m = ' + str(m) + ', corr = ' + 
    str(np.corrcoef( Qrec.reshape(-1), (Qs[m][np.ix_(idx1,idx2)]).reshape(-1) )[0,1])) )
    
    if mmap:
        del Q
        
    plt.xlabel('true covs')
    plt.ylabel('est. covs')
plt.show()
    
plt.figure(figsize=(20,10))
plt.plot(traces[0])
plt.xlabel('iteration count')
plt.ylabel('target loss')
plt.show()

if p <= 1000:
    plot_outputs_l2_gradient_test(pars_true=pars_true, pars_init=pars_init, 
                                  pars_est=pars_est, k=k, l=l, Qs=Qs, 
                                       Qs_full=Qs, Om=Om, traces = traces,
                                       linearity=linearity, idx_grp = idx_grp, co_obs = co_obs, 
                                       if_flip = True, m = 0)

In [ ]:
X_s = SSID_Hankel_loss.s_X_l2_Hankel_fully_obs(pars_true['C'], pars_true['R'], Qs, k, l, idx_grp, co_obs)
AmPi = X_s[:,m].reshape(n,n)

plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
plt.title('X gradients')
plt.subplot(1,3,2)
plt.imshow(AmPi, interpolation='none')
plt.title('X solved')
plt.subplot(1,3,3)
plt.plot(AmPi.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
plt.hold(True)
plt.plot([np.amin(AmPi),np.amax(AmPi)] , [np.amin(AmPi),np.amax(AmPi)], 'r')
plt.title('solving for X vs. gradients for X')
plt.xlabel('X solved')
plt.ylabel('X gradients')
plt.show()

AmPi_emp = np.cov(x[:,m:m-(k+l)], x[:,:-(k+l)])[:n, n:]
plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
plt.title('X gradients')
plt.subplot(1,3,2)
plt.imshow(AmPi_emp, interpolation='none')
plt.title('X empirical')
plt.subplot(1,3,3)
plt.plot(AmPi.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
plt.hold(True)
plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
plt.title('solving for X vs. ground-truth X')
plt.xlabel('X empirical')
plt.ylabel('X gradients')
plt.show()

plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
plt.title('X gradients')
plt.subplot(1,3,2)
plt.imshow(np.linalg.matrix_power(pars_true['A'],m).dot(pars_true['Pi']), interpolation='none')
plt.title('X ground-truth')
plt.subplot(1,3,3)
plt.plot(np.linalg.matrix_power(pars_true['A'],m).dot(pars_true['Pi']).reshape(-1), 
         X[m*n:(m+1)*n,:].reshape(-1), '.')
plt.hold(True)
plt.plot([np.amin(AmPi),np.amax(AmPi)] , [np.amin(AmPi),np.amax(AmPi)], 'r')
plt.title('gradients for X vs. ground-truth X')
plt.xlabel('X gradients')
plt.ylabel('X ground-truth')
plt.show()



In [ ]:
m = 0
X = np.zeros( ((k+l)*n, n) )
AmPi_emp = np.cov(x[:,m:m-(k+l)], x[:,:-(k+l)])[:n, n:]
#X[m*n:(m+1)*n,:] = AmPi.copy()
max_iter = 50
out = np.zeros(max_iter)

plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
plt.subplot(1,3,2)
plt.imshow(AmPi_emp, interpolation='none')
plt.subplot(1,3,3)
plt.plot(AmPi_emp.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
plt.hold(True)
plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
plt.show()
print(np.sum( (X[m*n:(m+1)*n,:])**2 ) )

batch_size = 1000

for i in range(max_iter):
    print(i)
    for t in range(1000//batch_size):
        X -= 0.0000001 * SSID_Hankel_loss.g_X_l2_Hankel_fully_obs(
                                                      pars_true['C'], X, pars_true['R'], y, 
                                                  ts = np.random.permutation( (T-(k+l)) )[:batch_size], 
                                                  ms = m * np.ones(batch_size) )
    out[i] = np.sum( (X[m*n:(m+1)*n,:] - AmPi_emp)**2 )
    
    if np.mod(i, 5) == 0:
        plt.figure(figsize=(15,7))
        plt.subplot(1,3,1)
        plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
        plt.subplot(1,3,2)
        plt.imshow(AmPi_emp, interpolation='none')
        plt.subplot(1,3,3)
        plt.plot(AmPi_emp.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
        plt.hold(True)
        plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
        plt.show()
        print(np.sum( (X[m*n:(m+1)*n,:])**2 ) )
    
plt.plot(out)
plt.show()



In [ ]:
m = 0
X = np.zeros( ((k+l)*n, n) )
AmPi_emp = np.cov(x[:,m:m-(k+l)], x[:,:-(k+l)])[:n, n:]
#X[m*n:(m+1)*n,:] = AmPi.copy()
max_iter = 50
out = np.zeros(max_iter)

plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
plt.subplot(1,3,2)
plt.imshow(AmPi_emp, interpolation='none')
plt.subplot(1,3,3)
plt.plot(AmPi_emp.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
plt.hold(True)
plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
plt.show()
print(np.sum( (X[m*n:(m+1)*n,:])**2 ) )

batch_size = 10

for i in range(max_iter):
    print(i)
    for t in range(1000//batch_size):
        X -= 0.0000001 * SSID_Hankel_loss.g_X_l2_Hankel_fully_obs(
                                                      pars_true['C'], X, pars_true['R'], y, 
                                                  ts = np.random.permutation( (T-(k+l)) )[:batch_size], 
                                                  ms = m * np.ones(batch_size) )
    out[i] = np.sum( (X[m*n:(m+1)*n,:] - AmPi_emp)**2 )
    
    if np.mod(i, 5) == 0:
        plt.figure(figsize=(15,7))
        plt.subplot(1,3,1)
        plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
        plt.subplot(1,3,2)
        plt.imshow(AmPi_emp, interpolation='none')
        plt.subplot(1,3,3)
        plt.plot(AmPi_emp.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
        plt.hold(True)
        plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
        plt.show()
        print(np.sum( (X[m*n:(m+1)*n,:])**2 ) )
    
plt.plot(out)
plt.show()



In [ ]:
m = 0
X = np.zeros( ((k+l)*n, n) )
AmPi_emp = np.cov(x[:,m:m-(k+l)], x[:,:-(k+l)])[:n, n:]
#X[m*n:(m+1)*n,:] = AmPi.copy()
max_iter = 50
out = np.zeros(max_iter)

plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
plt.subplot(1,3,2)
plt.imshow(AmPi_emp, interpolation='none')
plt.subplot(1,3,3)
plt.plot(AmPi_emp.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
plt.hold(True)
plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
plt.show()
print(np.sum( (X[m*n:(m+1)*n,:])**2 ) )

batch_size = 1

for i in range(max_iter):
    print(i)
    for t in range(1000//batch_size):
        X -= 0.0000001 * SSID_Hankel_loss.g_X_l2_Hankel_fully_obs(
                                                      pars_true['C'], X, pars_true['R'], y, 
                                                  ts = np.random.permutation( (T-(k+l)) )[:batch_size], 
                                                  ms = m * np.ones(batch_size) )
    out[i] = np.sum( (X[m*n:(m+1)*n,:] - AmPi_emp)**2 )
    
    if np.mod(i, 5) == 0:
        plt.figure(figsize=(15,7))
        plt.subplot(1,3,1)
        plt.imshow(X[m*n:(m+1)*n,:], interpolation='none')
        plt.subplot(1,3,2)
        plt.imshow(AmPi_emp, interpolation='none')
        plt.subplot(1,3,3)
        plt.plot(AmPi_emp.reshape(-1), X[m*n:(m+1)*n,:].reshape(-1), '.')
        plt.hold(True)
        plt.plot([np.amin(AmPi_emp),np.amax(AmPi_emp)] , [np.amin(AmPi_emp),np.amax(AmPi_emp)], 'r')
        plt.show()
        print(np.sum( (X[m*n:(m+1)*n,:])**2 ) )
    
plt.plot(out)
plt.show()



# just one more turn...

In [ ]:
from scipy.io import savemat # store results for comparison with Matlab code   

os.chdir('../fits/')

save_file = 'test_time_Tinf_p10000n100r1'


np.savez(save_file, 
         pars_init=pars_init,
         pars_true=pars_true, 
         pars_est=pars_est,
         #y=y,
         p=p,
         n=n,
         T=T,
         k=k,
         l=l,
         traces=traces,
         batch_size=batch_size,
         linearity=linearity)  

# Oops... (recovering broken run)